<a href="https://colab.research.google.com/github/neerajguleria1/driver-drowsiness-detection/blob/main/Untitled17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%%writefile decision_engine.py
class DecisionEngine:
  def decide(self, stable_state:str, trend:str, ema_risk:int)-> dict:

    reasons=[]

    if ema_risk>=85:
      reasons.append("EMA risk extrmely high")
      return self._decision(
          action="Stop vehicle immediately",
          severity="CRITICAL",
          message="Driver in dangerous condition",
          resons=reasons
      )

    if ema_risk>=70 and trend=="RISING":
      reasons.extend(["High EMA risk","Risk trending upward"])
      return self._decision(
          action="Recommended immediate break",
          severity="HIGH",
          message="Driver in dangerous condition",
          reasons=reasons
      )

      if 50<=ema_risk<70:
        reasons.append("Moderate EMA risk")

        if trend=="RISING":
          reasons.append("Upward trend detected")
          return self._decision(
              action="Recommended immediate break",
              severity="MEDIUM",
              message="Driver in moderate condition",
              reasons=reasons
          )
        return self._decision(
                action="Suggest short rest",
                severity="MEDIUM",
                message="Risk elevated but stable",
                reasons=reasons
            )


        if ema_risk < 50:
            reasons.append("Low EMA risk")

            if trend == "RISING":
                reasons.append("Early warning: upward trend")
                return self._decision(
                    action="Monitor closely",
                    severity="LOW",
                    message="Risk low but increasing",
                    reasons=reasons
                )

            return self._decision(
                action="Continue driving",
                severity="LOW",
                message="Driver condition stable",
                reasons=reasons
            )


        return self._decision(
            action="Monitor system",
            severity="UNKNOWN",
            message="Insufficient decision data",
            reasons=["Fallback triggered"]
        )
def _decision(self,action,severity,message,reasons):
  return{
      "action":action,
      "severity":severity,
      "message":message,
      "reasons":reasons
  }

Overwriting decision_engine.py


In [5]:
%%writefile temporal_risk.py
import numpy as np
from collections import deque

class TemporalRiskTracker:
    def __init__(self, window=5, alpha=0.3): # window=5 means how many pask risk values to remember
        self.window = window
        self.alpha = alpha
        self.risk_history = deque(maxlen=window)
        self.ema_risk = None
        self.current_state = "SAFE"

    def update(self, risk_score: int) -> dict:
        self.risk_history.append(risk_score)
        history = np.array(self.risk_history)# covert deque to numpy array makes math opeartion easy

        sma = int(history.mean())

        if self.ema_risk is None:
            self.ema_risk = risk_score # for fist value intializes ema because it has not past values
        else:
            self.ema_risk = int(
                self.alpha * risk_score +
                (1 - self.alpha) * self.ema_risk
            )

        trend = self._compute_trend(history)
        stable_state = self._stable_state(self.ema_risk)

        return {
            "current_risk": risk_score,
            "sma_risk": sma,
            "ema_risk": self.ema_risk,
            "trend": trend,
            "stable_state": stable_state
        }

    def _compute_trend(self, history: np.ndarray) -> str:
        if len(history) < 3:
            return "INSUFFICIENT_DATA"

        slope = np.polyfit(range(len(history)), history, 1)[0]

        if slope > 2:
            return "RISING"
        elif slope < -2:
            return "FALLING"
        else:
            return "STABLE"

    def _stable_state(self, risk: int) -> str:
        if self.current_state == "SAFE" and risk > 60:
            self.current_state = "CRITICAL"
        elif self.current_state == "CRITICAL" and risk < 40:
            self.current_state = "SAFE"

        return self.current_state

Writing temporal_risk.py


In [4]:
%%writefile cognitive_engine.py

"""
Cognitive Risk Engine

Purpose:
- Convert ML drowsiness probability into a explainable rish score
- Focus ML output with physocological + behavioural signals.
- Provide stable, decision-ready rish states
"""

import numpy as np
import pandas as pd

class CognitiveRiskEngine:
  REQUIRED_COLUMNS=[
      "Fatigue",
      "Alertness",
      "prev_alertness",
      "HR",
      "Speed",
      "speed_change"
  ]
  def __init__(self):
    self.prev_risk=0.0

  def compute(self,df:pd.DataFrame,ml_prob:float)->dict:
    missing=set(self.REQUIRED_COLUMNS) - set(df.columns)
    if missing:
      raise ValueError(f"Missing required input columns: {missing}")

    fatigue=df["Fatigue"].iloc[0] # iloc[0] give me the the actual number from the first row, not the whole column
    alertness=df["Alertness"].iloc[0]
    prev_alertness=df["prev_alertness"].iloc[0]
    hr=df["HR"].iloc[0]
    speed=df["Speed"].iloc[0]
    speed_change=df["speed_change"].iloc[0]

    fatigue_score=np.clip(fatigue/10,0,1)
    alertness_low=np.clip(1-alertness,0,1)

    alertness_drop=1.0 if (prev_alertness-alertness) > 0.2 else 0.0

    hr_stress=np.clip((hr-90)/30,0,1)
    speed_instability=np.clip(speed_change/30,0,1)


    raw_risk = (
        0.35*ml_prob
        +0.20*fatigue_score
        +0.15*alertness_low
        +0.10*alertness_drop
        +0.10*hr_stress
        +0.10*speed_instability
    )

    smoothed_risk=0.7*self.prev_risk +0.3*raw_risk
    self.prev_risk=smoothed_risk

    risk_score=int(np.clip(smoothed_risk*100,0,100))

    if risk_score<30:
      state="SAFE"
      action="No alert"
    elif risk_score<60:
      state="CAUTION"
      action="Audio warning"
    else:
      state="CRITICAL"
      action="Audio + Visual + Haptic alert"


    return {
        "risk_score":risk_score,
        "state":state,
        "recommended_action":action,
        "explanation":{
            "ml_probability":round(ml_prob,2),
            "fatigue_score":round(fatigue_score,2),
            "alertness_low":round(alertness_low,2),
            "hr_stress":round(hr_stress,2),
            "speed_instability":round(speed_instability,2)
        }
    }


Writing cognitive_engine.py


In [10]:
import joblib
import pandas as pd

from cognitive_engine import CognitiveRiskEngine
from temporal_risk import TemporalRiskTracker
from decision_engine import DecisionEngine

from google.colab import files


PIPELINE_PATH = "final_driver_drowsiness_pipeline.pkl"
pipeline = joblib.load(PIPELINE_PATH)


cognitive = CognitiveRiskEngine()
temporal = TemporalRiskTracker()
decision_engine = DecisionEngine()


samples = [
    {"Speed": 70, "Alertness": 0.7, "Seatbelt": 1, "HR": 85,  "Fatigue": 4, "speed_change": 13, "prev_alertness": 0.75},
    {"Speed": 65, "Alertness": 0.6, "Seatbelt": 1, "HR": 95,  "Fatigue": 5, "speed_change": 9,  "prev_alertness": 0.70},
    {"Speed": 60, "Alertness": 0.5, "Seatbelt": 1, "HR": 102, "Fatigue": 6, "speed_change": 6,  "prev_alertness": 0.60},
    {"Speed": 58, "Alertness": 0.45,"Seatbelt": 1, "HR": 108, "Fatigue": 7, "speed_change": 5,  "prev_alertness": 0.55},
    {"Speed": 55, "Alertness": 0.4, "Seatbelt": 1, "HR": 112, "Fatigue": 8, "speed_change": 4,  "prev_alertness": 0.50},
]


for i, row in enumerate(samples, start=1):

    df = pd.DataFrame([row])

    ml_prob = pipeline.predict_proba(df)[0][1]

    cognitive_output = cognitive.compute(df, ml_prob)

    temporal_output = temporal.update(cognitive_output["risk_score"])

    decision = decision_engine.decide(
        stable_state=temporal_output["stable_state"],
        trend=temporal_output["trend"],
        ema_risk=temporal_output["ema_risk"]
    )


    print(f"\n===== FRAME {i} =====")
    print("ML Probability:", round(ml_prob, 3))
    print("Cognitive:", cognitive_output)
    print("Temporal:", temporal_output)
    print("Decision:", decision)



===== FRAME 1 =====
ML Probability: 0.457
Cognitive: {'risk_score': 9, 'state': 'SAFE', 'recommended_action': 'No alert', 'explanation': {'ml_probability': np.float64(0.46), 'fatigue_score': np.float64(0.4), 'alertness_low': np.float64(0.3), 'hr_stress': np.float64(0.0), 'speed_instability': np.float64(0.43)}}
Temporal: {'current_risk': 9, 'sma_risk': 9, 'ema_risk': 9, 'trend': 'INSUFFICIENT_DATA', 'stable_state': 'SAFE'}
Decision: None

===== FRAME 2 =====
ML Probability: 0.476
Cognitive: {'risk_score': 18, 'state': 'SAFE', 'recommended_action': 'No alert', 'explanation': {'ml_probability': np.float64(0.48), 'fatigue_score': np.float64(0.5), 'alertness_low': np.float64(0.4), 'hr_stress': np.float64(0.17), 'speed_instability': np.float64(0.3)}}
Temporal: {'current_risk': 18, 'sma_risk': 13, 'ema_risk': 11, 'trend': 'INSUFFICIENT_DATA', 'stable_state': 'SAFE'}
Decision: None

===== FRAME 3 =====
ML Probability: 0.378
Cognitive: {'risk_score': 24, 'state': 'SAFE', 'recommended_action': 